In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
HEADER = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
url_list = [
    'https://movie.douban.com/people/98909222/wish?start=0&sort=time&rating=all&filter=all&mode=list',
    'https://movie.douban.com/people/98909222/wish?start=30&sort=time&rating=all&filter=all&mode=list'
]

In [3]:
def get_movie_info(movie_url):
    r_movie = requests.get(movie_url, headers=HEADER)
    soup_movie = BeautifulSoup(r_movie.text, 'html.parser')
    return {
        'name': str(soup_movie.find_all('span', {'property': 'v:itemreviewed'})[0].contents[0]),
        'year': int(soup_movie.find_all('span', {'class': 'year'})[0].contents[0].replace('(', '').replace(')', '')),
        'rating': float(soup_movie.find_all('strong', {'class': 'll rating_num', 'property': 'v:average'})[0].contents[0]),
        'votes': int(soup_movie.find_all('span', {'property': 'v:votes'})[0].contents[0])
    }

In [4]:
def get_page_info(url):
    r_page = requests.get(url, headers=HEADER)
    soup_page = BeautifulSoup(r_page.text, 'html.parser')  # 'lxml'
    movies = soup_page.find_all('div', {'class': 'title'})
    for i in range(0, len(movies)):
        movies_list.append(get_movie_info(movies[i].contents[1].attrs['href']))
        # print('No. ' + str(i))
    del i

In [5]:
movies_list = []
for i in range(0, len(url_list)):
    get_page_info(url_list[i])
del i

In [6]:
movies_df = pd.DataFrame(data=movies_list)
movies_df['rating_year'] = movies_df['rating'] * movies_df['year']
movies_df['rating_votes'] = movies_df['rating'] * movies_df['votes']
movies_df['rating_year_votes'] = movies_df['rating'] * movies_df['year'] * movies_df['votes']

In [7]:
movies_df.sort_values('rating_year_votes', ascending=False)

,name,year,rating,votes,rating_year,rating_votes,rating_year_votes
28,头号玩家 Ready Player One,2018,8.7,1032782,17556.6,8985203.4,1.813214e+10
50,教父 The Godfather,1972,9.3,682664,18339.6,6348775.2,1.251978e+10
49,教父2 The Godfather: Part Ⅱ,1974,9.2,378343,18160.8,3480755.6,6.871012e+09
42,社交网络 The Social Network,2010,8.1,365719,16281.0,2962323.9,5.954271e+09
17,雪国列车 설국열차,2013,7.5,332569,15097.5,2494267.5,5.020960e+09
15,E.T. 外星人 E.T.: The Extra-Terrestrial,1982,8.6,264266,17045.2,2272687.6,4.504467e+09
47,教父3 The Godfather: Part III,1990,8.9,252499,17711.0,2247241.1,4.472010e+09
43,华尔街之狼 The Wolf of Wall Street,2013,7.9,274491,15902.7,2168478.9,4.365148e+09
9,侏罗纪世界2 Jurassic World: Fallen Kingdom,2018,6.7,314893,13520.6,2109783.1,4.257542e+09
11,绿里奇迹 The Green Mile,1999,8.8,205419,17591.2,1807687.2,3.613567e+09
